In [35]:
import pandas as pd
import requests
import bs4 as bfs
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string
from textblob import TextBlob
import csv
import numpy as np

In [36]:
!pip install openpyxl

In [37]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manju\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
df = pd.read_excel('Input ..xlsx',index_col=0)
df

,URL
URL_ID,
37.0,https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/
38.0,https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/
39.0,https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/
40.0,https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/
41.0,https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/
...,...
146.0,https://insights.blackcoffer.com/blockchain-for-payments/
147.0,https://insights.blackcoffer.com/the-future-of-investing/
148.0,https://insights.blackcoffer.com/big-data-analytics-in-healthcare/


In [41]:
li = [url for url in df['URL']]
li

['https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/',
 'https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/',
 'https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/',
 'https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/',
 'https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/',
 'https://insights.blackcoffer.com/man-and-machines-together-machines-are-more-diligent-than-humans-blackcoffe/',
 'https://insights.blackcoffer.com/in-future-or-in-upcoming-years-humans-and-machines-are-going-to-work-together-in-every-field-of-work/',
 'https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/',
 'https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/',
 'https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/',
 'https://insights.blackcoffer.com/how-to-protect-fu

In [ ]:
text = []
for url in li:
  text.append(requests.get(url,headers={"User-Agent": "XY"}))

In [ ]:
for i in range(len(text)):
  text[i] = bfs.BeautifulSoup(text[i].content,'html.parser')

In [ ]:
articles = []
for text in text:
  articles.append(text.find(attrs= {"class":"td-post-content"}).text)

In [ ]:
for i in range(len(articles)):
  articles[i]= articles[i].replace('\n','')

In [64]:
stop_words = list(set(stopwords.words('english')))


In [65]:
sentences = []
for article in articles:
  sentences.append(len(sent_tokenize(article)))

In [66]:
cleaned_articles = [' ']*len(articles)

In [67]:
for i in range(len(articles)):
  for w in stop_words:
    cleaned_articles[i]= articles[i].replace(' '+w+' ',' ').replace('?','').replace('.','').replace(',','').replace('!','')
    #? ! , .

In [68]:
words = []
for article in articles:
  words.append(len(word_tokenize(article)))

In [69]:
words_cleaned = []
for article in cleaned_articles:
  words_cleaned.append(len(word_tokenize(article)))

In [127]:
positive_words = pd.read_csv("positive-words (1).txt")

In [131]:
n = open("negative-words (2).txt","r+")

In [132]:
negative_words = pd.read_csv(n)

In [135]:
negative_words = negative_words.fillna(0)

In [136]:
len(positive_words)

2005

In [137]:
len(negative_words)

4782

In [138]:
positive_score = [0]*len(articles)
for i in range(len(articles)):
  for word in positive_words:
    for letter in cleaned_articles[i].lower().split(' '):
      if letter==word:
        positive_score[i]+=1

In [139]:
negative_score = [0]*len(articles)
for i in range(len(articles)):
  for word in negative_words:
    for letter in cleaned_articles[i].upper().split(' '):
      if letter==word:
        negative_score[i]+=1

In [140]:
words_cleaned = np.array(words_cleaned)
sentences = np.array(sentences)

In [149]:
df = pd.DataFrame()

In [150]:
df['POSITIVE SCORE'] = positive_score
df['NEGATIVE SCORE'] = negative_score

In [151]:
df['POLARITY SCORE'] = (df['POSITIVE SCORE']-df['NEGATIVE SCORE'])/ ((df['POSITIVE SCORE'] +df['NEGATIVE SCORE']) + 0.000001)

In [152]:
df['SUBJECTIVITY SCORE'] = (df['POSITIVE SCORE'] + df['NEGATIVE SCORE'])/( (words_cleaned) + 0.000001)

In [153]:
df['AVG SENTENCE LENGTH'] = np.array(words)/np.array(sentences)

In [154]:
complex_words = []
sylabble_counts = []

In [155]:
for article in articles:
  sylabble_count=0
  d=article.split()
  ans=0
  for word in d:
    count=0
    for i in range(len(word)):
      if(word[i]=='a' or word[i]=='e' or word[i] =='i' or word[i] == 'o' or word[i] == 'u'):
           count+=1
#            print(words[i])
      if(i==len(word)-2 and (word[i]=='e' and word[i+1]=='d')):
        count-=1;
      if(i==len(word)-2 and (word[i]=='e' and word[i]=='s')):
        count-=1;
    sylabble_count+=count    
    if(count>2):
        ans+=1
  sylabble_counts.append(sylabble_count)
  complex_words.append(ans)         

In [156]:
df['PERCENTAGE OF COMPLEX WORDS'] = np.array(complex_words)/np.array(words)
df['FOG INDEX'] = 0.4 * (df['AVG SENTENCE LENGTH'] + df['PERCENTAGE OF COMPLEX WORDS'])
df['AVG NUMBER OF WORDS PER SENTENCES'] = df['AVG SENTENCE LENGTH']
df['COMPLEX WORD COUNT'] = complex_words
df['WORD COUNT'] = words
df['SYLLABLE PER WORD'] = np.array(sylabble_counts)/np.array(words)
total_characters = []
for article in articles:
  characters = 0
  for word in article.split():
    characters+=len(word)
  total_characters.append(characters)  
personal_nouns = []
personal_noun =['I', 'we','my', 'ours','and' 'us','My','We','Ours','Us','And'] 
for article in articles:
  ans=0
  for word in article:
    if word in personal_noun:
      ans+=1
  personal_nouns.append(ans)
df['PERSONAL PRONOUN'] = personal_nouns
#as the all pronouns were cleared when clearing the stop words.
df['AVG WORD LENGTH'] = np.array(total_characters)/np.array(words)
df

,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCES,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,AVG WORD LENGTH
0,0,0,0.0,0.0,26.129870,0.301193,10.572425,26.129870,606,2012,1.859841,39,5.160040
1,0,0,0.0,0.0,20.525000,0.195493,8.288197,20.525000,321,1642,1.542631,37,4.302680
2,0,0,0.0,0.0,22.552941,0.279082,9.132809,22.552941,535,1917,1.811163,31,4.912363
3,0,0,0.0,0.0,19.284211,0.199782,7.793597,19.284211,366,1832,1.581332,74,4.428493
4,0,0,0.0,0.0,25.050633,0.210207,10.104336,25.050633,416,1979,1.643759,36,4.612936
5,0,0,0.0,0.0,23.296875,0.212609,9.403794,23.296875,317,1491,1.625084,17,4.498994
6,0,0,0.0,0.0,18.688889,0.214031,7.561168,18.688889,180,841,1.630202,14,4.564804


In [157]:
articles

['\n\nIntroduction\n“If anything kills over 10 million people in the next few decades, it will be a highly infectious virus rather than a war. Not missiles but microbes.” Bill Gates’s remarks at a TED conference in 2014, right after the world had avoided the Ebola outbreak. When the new, unprecedented, invisible virus hit us, it met an overwhelmed and unprepared healthcare system and oblivious population. This public health emergency demonstrated our lack of scientific consideration and underlined the alarming need for robust innovations in our health and medical facilities. For the past few years, artificial intelligence has proven to be of tangible potential in the healthcare sectors, clinical practices, translational medical and biomedical research.\nAfter the first case was detected in China on December 31st 2019, it was an AI program developed by BlueDot that alerted the world about the pandemic. It was quick to realise AI’s ability to analyse large chunks of data could help in de